In [1]:
# %% [설치 - 필요시만 실행]
# 주피터 커널에 패키지가 없다면 아래 주석을 풀고 한 번만 실행하세요.
# %pip install requests beautifulsoup4 lxml pandas urllib3


In [2]:
# %% [임포트 & 전역 설정]
from __future__ import annotations
import os, re, time
from dataclasses import dataclass, asdict
from typing import Optional, List, Dict
from urllib.parse import urljoin, urlparse, parse_qs, quote

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup, NavigableString, Tag
import pandas as pd

# 대상 목록 URL (마루아트센터 - 현재전시)
LIST_URL = "https://maruartcenter.co.kr/default/exhibit/exhibit01.php?sub=01"

# 파서 우선순위 (lxml이 설치되어 있으면 사용 권장)
PARSER = "lxml"  # 설치가 안 되어 있으면 html.parser로 자동 폴백

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/124.0 Safari/537.36"
    ),
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8",
}
TIMEOUT = 15
SLEEP_BETWEEN = 0.3   # 요청 간 짧은 대기 (서버 예의상)
MAX_PAGES = 1         # 목록 페이징 개수
IMG_DIR = "maru_images"  # 이미지 저장 루트
DOWNLOAD_IMAGES = False   # True로 두면 이미지 파일 다운로드
MAX_IMGS_PER_POST = None  # None = 제한 없음, 정수 = 최대 저장 개수


In [3]:
# %% [세션/유틸 - 요청 세션 만들기]
def get_session() -> requests.Session:
    """재시도/커넥션풀 설정된 세션 반환"""
    s = requests.Session()
    s.headers.update(HEADERS)
    retry = Retry(
        total=3,
        backoff_factor=0.4,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET", "HEAD"],
    )
    s.mount("http://", HTTPAdapter(max_retries=retry, pool_connections=8, pool_maxsize=16))
    s.mount("https://", HTTPAdapter(max_retries=retry, pool_connections=8, pool_maxsize=16))
    return s


In [4]:
# %% [세션/유틸 - Soup 생성기]
def _make_soup(html: str) -> BeautifulSoup:
    try:
        return BeautifulSoup(html, PARSER)
    except Exception:
        return BeautifulSoup(html, "html.parser")

def _get_soup(url: str, s: requests.Session, *, referrer: Optional[str] = None) -> BeautifulSoup:
    headers = dict(HEADERS)
    if referrer:
        headers["Referer"] = referrer  # 핫링크/출처 검증 회피
    r = s.get(url, headers=headers, timeout=TIMEOUT)
    # 인코딩 보정
    if not r.encoding or r.encoding.lower() in ("iso-8859-1", "ascii"):
        r.encoding = r.apparent_encoding or r.encoding
    return _make_soup(r.text)


In [5]:
# %% [텍스트/라벨 유틸]
def _clean_text(txt: str) -> str:
    if not txt:
        return ""
    txt = txt.replace("\xa0", " ")
    txt = re.sub(r"\s+", " ", txt).strip()
    return txt

def _norm(s: str) -> str:
    """라벨 비교용 정규화: 공백/콜론 등 제거"""
    if not s:
        return ""
    s = s.replace("\xa0", " ")
    s = re.sub(r"\s+", "", s)
    s = s.replace(":", "").replace("：", "")
    return s

def _abs_url(base: str, u: str) -> str:
    return urljoin(base, u)


In [6]:
# %% [목록 수집]
def list_current_detail_urls(list_url: str = LIST_URL, *, max_pages: int = MAX_PAGES) -> List[str]:
    """현재전시 목록 페이지에서 상세(read_form) 링크를 수집."""
    out: List[str] = []
    seen = set()
    with get_session() as s:
        for page in range(1, max_pages + 1):
            url = list_url
            if page > 1:
                sep = '&' if '?' in url else '?'
                url = f"{url}{sep}com_board_page={page}"
            soup = _get_soup(url, s)
            # read_form + com_board_idx가 들어간 상세 링크만 추출
            for a in soup.select('a[href*="exhibit01.php"][href*="read_form"], a[href*="com_board_idx="]'):
                href = a.get("href")
                if not href:
                    continue
                u = urljoin(list_url, href)
                if "read_form" in u and "com_board_idx=" in u:
                    if u not in seen:
                        seen.add(u)
                        out.append(u)
            time.sleep(SLEEP_BETWEEN)
    return out


In [7]:
# %% [패턴 정의]
_PERIOD_PATTERNS = [
    re.compile(r"(?P<s>\d{4}\.\s*\d{1,2}\.\s*\d{1,2})\s*[-–~]\s*(?P<e>\d{1,2}\.\s*\d{1,2})"),
    re.compile(r"(?P<s>\d{4}\.\s*\d{1,2}\.\s*\d{1,2})\s*[-–~]\s*(?P<e>\d{4}\.\s*\d{1,2}\.\s*\d{1,2})"),
    re.compile(r"기간\s*[:：]?\s*(?P<s>\d{4}\.\s*\d{1,2}\.\s*\d{1,2})\s*[-–~]\s*(?P<e>\d{1,2}\.\s*\d{1,2}(?:\.\s*\d{1,2})?)"),
]

_SECTION_LABELS = [
    ("전시설명", re.compile(r"^\s*\[?전시\s*설명\]?\s*$")),
    ("전시서문", re.compile(r"^\s*\[?전시\s*서문\]?\s*$")),
    ("작가노트", re.compile(r"^\s*\[?작가\s*노트\]?\s*$")),
    ("작가의 글", re.compile(r"^\s*\[?작가의\s*글\]?\s*$")),
]


In [8]:
# %% [테이블 필드 추출 - 견고 버전]
def _extract_table_field(soup: BeautifulSoup, label_keywords: List[str]) -> Optional[str]:
    """테이블에서 라벨(TD)과 값(TD)을 매칭해 값 추출."""
    keys = [_norm(k) for k in label_keywords]

    # (1) 클래스가 있는 경우 우선 이용
    for td in soup.select("td.board_bgcolor"):
        left = _clean_text(td.get_text(" "))
        if _norm(left) in keys or any(k in _norm(left) for k in keys):
            tr = td.find_parent("tr")
            if tr:
                cand = tr.find("td", class_="board_desc")
                if cand:
                    val = _clean_text(cand.get_text(" "))
                    if val:
                        return val
                tds = tr.find_all("td")
                try:
                    i = tds.index(td)
                    for j in range(i+1, len(tds)):
                        val = _clean_text(tds[j].get_text(" "))
                        if val:
                            return val
                except ValueError:
                    pass

    # (2) 모든 tr을 훑으며 인접 td 쌍 스캔
    for tr in soup.select("tr"):
        tds = tr.find_all("td")
        if len(tds) < 2:
            continue
        for i in range(len(tds) - 1):
            left = _clean_text(tds[i].get_text(" "))
            right = _clean_text(tds[i+1].get_text(" "))
            if not left and right:
                left, right = right, left  # 좌우가 바뀐 레이아웃 대비
            ln = _norm(left)
            if ln in keys or any(k in ln for k in keys):
                if right:
                    return right
    return None


In [9]:
# %% [제목/기간 추출기]
def _extract_title(soup: BeautifulSoup) -> Optional[str]:
    # 표에서 '제목' 우선 추출
    t = _extract_table_field(soup, ["제목", "전시명", "Title"])
    if t:
        return t
    # og:title
    m = soup.find("meta", attrs={"property": "og:title"})
    if m and m.get("content"):
        t = _clean_text(m["content"]) 
        if t:
            return t
    # 헤딩/클래스 후보
    candidates = []
    for sel in ["h1", "h2", ".tit", ".title", ".subject", ".board_tit", ".view_tit"]:
        for el in soup.select(sel):
            txt = _clean_text(el.get_text(" "))
            if txt:
                candidates.append((len(txt), txt))
    if candidates:
        candidates.sort(reverse=True)
        return candidates[0][1]
    # <title> 폴백
    if soup.title and soup.title.string:
        return _clean_text(soup.title.string)
    return None

def _extract_period_from_table_or_text(soup: BeautifulSoup) -> Optional[str]:
    # 표에서 '기간' 우선 추출
    p = _extract_table_field(soup, ["기간", "전시기간", "전시 일정", "DATE"])
    if p:
        return p
    # 본문 텍스트에서 패턴 탐지
    text = _clean_text(soup.get_text("\n"))
    for pat in _PERIOD_PATTERNS:
        m = pat.search(text)
        if m:
            s = _clean_text(m.group("s"))
            e = _clean_text(m.group("e"))
            return f"{s} - {e}"
    # 라인에 '기간' 포함 시 그 라인 반환
    for line in text.splitlines():
        if "기간" in line:
            line = _clean_text(line)
            if len(line) > 3:
                return line
    return None


In [10]:
# %% [본문 스코프 & 푸터 필터 헬퍼]

# 본문 컨테이너 후보 (페이지에 따라 다를 수 있어 넓게 지정)
_CONTENT_SELECTORS = ".board_view, .view, .view_area, .view_con, .board, #board, .content, .editor, .viewDetail"

# 푸터/하단 영역 셀렉터 (제외)
_FOOTER_SELECTORS = "footer, #footer, .footer, .foot, .ft, .bottom, .site-info, address"

# 푸터 키워드/패턴 (라인 단위로 감지)
_FOOTER_PAT = re.compile(
    r"("
    r"개인정보(처리|취급)방침|이메일무단수집거부|오시는길|고객센터|회사\s*:|대표자|사업자|사업자등록|"
    r"주소\s*:|Tel\s*:|Fax\s*:|EMAIL\s*:|E-?mail\s*:|Copyright|COPYRIGHT|All\s+Right[s]?\s+Reserved"
    r")",
    re.IGNORECASE
)

def get_content_scope(soup: BeautifulSoup) -> Tag:
    """
    본문 컨테이너를 찾아 반환.
    - 후보 셀렉터 중 첫 번째를 사용
    - 찾지 못하면 문서 전체 반환하되, 이후 푸터 영역은 제거
    """
    scope = soup.select_one(_CONTENT_SELECTORS) or soup
    # scope 내부에서 푸터/하단 영역은 제거(있으면)
    for f in scope.select(_FOOTER_SELECTORS):
        f.decompose()
    return scope

def is_footer_text(text: str) -> bool:
    """한 줄이 푸터/연락처/저작권 안내에 해당하는지"""
    t = _clean_text(text)
    if not t:
        return False
    return bool(_FOOTER_PAT.search(t))

def trim_footer_tail(block_text: str) -> str:
    """
    블록 텍스트의 '끝부분'에서 푸터 패턴이 보이면 그 지점부터 잘라냄.
    (본문 끝에 붙은 연락처/카피라이트 꼬리를 제거)
    """
    if not block_text:
        return block_text
    lines = [l.rstrip() for l in block_text.splitlines()]
    cut = len(lines)
    for i, line in enumerate(lines):
        if is_footer_text(line):
            cut = i
            break
    cleaned = "\n".join(lines[:cut]).rstrip()
    # 과도한 빈줄 정리
    cleaned = re.sub(r"\n{3,}", "\n\n", cleaned)
    return cleaned


In [11]:
# %% [섹션 추출 헬퍼]

# 라벨 판별(대괄호 유/무 모두 허용)
_LABEL_RE = re.compile(r"^\s*\[?(전시\s*설명|전시\s*서문|작가\s*노트|작가의\s*글)\]?\s*$")

def _is_label_text(t: str) -> bool:
    return bool(_LABEL_RE.match(_clean_text(t)))

def _nearest_block(tag: Tag) -> Tag:
    cur = tag
    while cur and isinstance(cur, Tag) and cur.name.lower() not in {"p", "div", "li", "section", "article"}:
        cur = cur.parent
    return cur if isinstance(cur, Tag) else tag

def _collect_following_text(start: Tag) -> str:
    """
    라벨 블록 이후를 '문서 순회'로 수집.
    - 형제만 보지 않고 .find_all_next()로 다음 요소들을 따라가며 모음
    - 다음 섹션 경계(헤딩/새 라벨/푸터/수평선/분리선 라인)에서 중지
    - <br> 줄바꿈 보존, <strong>/<b>는 본문 허용
    """
    buf: List[str] = []
    started = False

    # 현재 블록 포함 이후를 순서대로 훑는다
    for el in start.find_all_next():
        # start 자신을 지나친 뒤부터 수집 시작
        if el is start:
            started = True
            continue
        if not started:
            continue

        # 푸터 영역이면 중지
        if isinstance(el, Tag):
            if el.name.lower() in {"footer", "address"}:
                break
            if el.select_one(_FOOTER_SELECTORS):
                break
            # 섹션 헤딩(h1~h4)이면 다음 섹션 시작으로 간주
            if el.name.lower() in {"h1", "h2", "h3", "h4"}:
                break
            # 수평선으로도 절단
            if el.name.lower() in {"hr"}:
                break

        # 텍스트 추출
        t = ""
        if isinstance(el, NavigableString):
            t = _clean_text(str(el))
        elif isinstance(el, Tag):
            # 글자 없는 컨테이너는 스킵 (이미 푸터/헤딩은 위에서 걸렀음)
            t = _clean_text(el.get_text("\n"))

        if not t:
            continue

        # 새 라벨 패턴을 만나면 종료
        if _is_label_text(t):
            break

        # 흔한 분리선(—, -, ▼, ▲ 등)을 만나면 종료
        if t.strip() in {"-", "–", "—", "▼", "▲"}:
            break

        buf.append(t)

        # 과다 수집 방지
        if sum(len(x) for x in buf) > 12000:
            break

    out = "\n".join(x for x in buf if x).strip()
    out = re.sub(r"\n{3,}", "\n\n", out)
    out = trim_footer_tail(out)  # 본문 끝의 푸터 꼬리 제거
    return out


def _extract_sections(soup: BeautifulSoup) -> Dict[str, str]:
    """
    전시설명/전시서문/작가노트/작가의 글 텍스트 추출
    - 문서 전체가 아닌 본문 스코프(get_content_scope)에서만 탐색
    - 푸터 영역은 사전 제거
    """
    root = get_content_scope(soup)
    textmap: Dict[str, str] = {}

    # (1) [라벨] 패턴 먼저
    for node in root.find_all(string=True):
        s = _clean_text(str(node))
        if not s:
            continue
        if _is_label_text(s):
            block = _nearest_block(node if isinstance(node, Tag) else node.parent)
            content = _collect_following_text(block)
            if content:
                label = _clean_text(s).strip("[]")
                textmap[label] = content

    # (2) 대괄호 없이 헤딩/볼드로만 라벨이 있는 경우
    if not textmap:
        for lab, pat in _SECTION_LABELS:
            for el in root.find_all(["h1", "h2", "h3", "strong", "b", "p", "div"]):
                t = _clean_text(el.get_text(" "))
                if pat.match(t):
                    content = _collect_following_text(el)
                    if content:
                        textmap[lab] = content

    # (3) 폴백: 본문에서 가장 긴 문단 (푸터 꼬리 제거)
    if not textmap:
        paragraphs = [
            _clean_text(p.get_text("\n"))
            for p in root.find_all(["p", "div"])
        ]
        paragraphs = [p for p in paragraphs if p and len(p) >= 40]
        paragraphs.sort(key=len, reverse=True)
        if paragraphs:
            textmap["본문"] = trim_footer_tail(paragraphs[0])

    return textmap


In [12]:
# %% [이미지 수집 & 다운로드]
_IMG_EXT = (".jpg", ".jpeg", ".png", ".gif", ".webp", ".avif")

# 불필요 이미지(아이콘/로고/네비게이션 버튼 등) 필터 규칙
_EXCLUDE_PATH_SUBSTR = [
    "/default/img/common/",          # 공통 아이콘/로고 경로
    "/img/common/",                  # 공통 이미지 경로
    "/component/board/board_10/list.gif",
    "/component/board/board_10/write.gif",
]
_EXCLUDE_NAME_EXACT = {
    "icon-phone.png", "icon-insta.png", "icon-blog.png", "icon-map.png",
    "icon-top.png", "logo.png", "logo-m.png", "logo-f.png",
}
_EXCLUDE_NAME_PREFIX = ("icon", "logo")  # 파일명이 icon*, logo* 로 시작하면 제외

def _norm_name_from_url(u: str) -> str:
    name = os.path.basename(urlparse(u).path)
    name = re.sub(r'^thumb-', '', name, flags=re.IGNORECASE)
    name = re.sub(r'_(\d+)x(\d+)(?=\.[A-Za-z0-9]+$)', '', name)
    return name.lower()

def dedupe_img_urls_by_key(img_urls: List[str]) -> List[str]:
    uniq, seen = [], set()
    for u in img_urls:
        key = _norm_name_from_url(u)
        if key and key not in seen:
            seen.add(key)
            uniq.append(u)
    return uniq

def _should_keep_image(u: str) -> bool:
    """아이콘/로고/네비게이션 버튼 등 불필요 이미지를 제외"""
    p = urlparse(u).path.lower()
    name = os.path.basename(p)
    # 경로 기반 제외
    for sub in _EXCLUDE_PATH_SUBSTR:
        if sub in p:
            return False
    # 파일명 정확 매칭 제외
    if name in _EXCLUDE_NAME_EXACT:
        return False
    # 접두사 기반 제외
    if name.startswith(_EXCLUDE_NAME_PREFIX):
        return False
    return True

def collect_image_urls(detail_url: str, soup: BeautifulSoup) -> List[str]:
    """상세 본문에서 이미지 URL을 절대경로로 수집 (lazy-src 포함)"""
    urls: List[str] = []
    scope = soup.select_one(".board_view, .view, .view_area, .view_con, .board, #board, .content, .editor") or soup
    for img in scope.find_all("img"):
        cand = None
        for attr in ("src", "data-src", "data-original", "data-lazy", "data-echo"):
            v = img.get(attr)
            if v and isinstance(v, str):
                cand = v
                break
        if not cand:
            continue
        u = _abs_url(detail_url, cand)
        # 확장자 필터(없어도 수집, 다만 확장자 있으면 체크) + 불필요 이미지 제외
        path = urlparse(u).path.lower()
        if ((not os.path.splitext(path)[1]) or path.endswith(_IMG_EXT)) and _should_keep_image(u):
            urls.append(u)
    # 중복 제거
    urls = dedupe_img_urls_by_key(urls)
    return urls

def _filename_from_url_or_headers(url: str, resp) -> str:
    base = os.path.basename(urlparse(url).path)
    if base:
        return base
    cd = resp.headers.get("Content-Disposition", "")
    m = re.search(r'filename\*?=(?:UTF-8\'\')?"?([^";]+)"?', cd)
    if m:
        return m.group(1)
    ctype = (resp.headers.get("Content-Type") or "").lower()
    if "png" in ctype: return "image.png"
    if "webp" in ctype: return "image.webp"
    return "image.jpg"

def download_images_from_urls(detail_url: str, img_urls: List[str], img_dir: str = IMG_DIR, max_imgs: Optional[int] = MAX_IMGS_PER_POST) -> List[str]:
    if not img_urls:
        return []
    img_urls = dedupe_img_urls_by_key(img_urls)
    qs = parse_qs(urlparse(detail_url).query)
    post_id = qs.get("com_board_idx", ["unknown"])[0]
    subdir = os.path.join(img_dir, re.sub(r"[^0-9A-Za-z_-]", "_", post_id))
    os.makedirs(subdir, exist_ok=True)

    saved: List[str] = []
    tried = 0
    with get_session() as s:
        _ = _get_soup(detail_url, s)  # 프리히트
        for u in img_urls:
            if max_imgs is not None and tried >= max_imgs:
                break
            tried += 1
            try:
                r = s.get(
                    u,
                    headers={**HEADERS, "Referer": detail_url, "Accept": "image/avif,image/webp,image/apng,image/*,*/*;q=0.8"},
                    timeout=TIMEOUT,
                    allow_redirects=True,
                )
                ctype = (r.headers.get("Content-Type") or "").lower()
                if r.status_code == 200 and r.content and "image" in ctype:
                    name = _filename_from_url_or_headers(u, r)
                    if "." not in os.path.basename(name):
                        if "png" in ctype: name += ".png"
                        elif "webp" in ctype: name += ".webp"
                        else: name += ".jpg"
                    base, ext = os.path.splitext(name)
                    final = os.path.join(subdir, name)
                    k = 1
                    while os.path.exists(final):
                        final = os.path.join(subdir, f"{base}_{k}{ext}")
                        k += 1
                    with open(final, "wb") as f:
                        f.write(r.content)
                    saved.append(final)
                else:
                    print(f"[이미지 응답 이상] {r.status_code} {u} (ctype={ctype})")
            except Exception as e:
                print(f"[이미지 실패] {u} -> {e}")
    return saved


In [13]:
# %% [데이터 모델 & 상세 파서]
@dataclass
class ExhibitRecord:
    url: str
    title: str
    period: str
    section_type: str
    section_text: str
    image_urls: List[str]
    saved_images: List[str]

def parse_detail(url: str, s: Optional[requests.Session] = None, *, download_images: bool = DOWNLOAD_IMAGES) -> ExhibitRecord:
    own = False
    if s is None:
        s = get_session()
        own = True
    try:
        soup = _get_soup(url, s, referrer=LIST_URL)
        title = _extract_title(soup) or ""
        period = _extract_period_from_table_or_text(soup) or ""
        sections = _extract_sections(soup)
        order = ["전시설명", "전시서문", "작가노트", "작가의 글", "본문"]
        section_type, section_text = "", ""
        for k in order:
            if k in sections and sections[k]:
                section_type, section_text = k, sections[k]
                break
        image_urls = collect_image_urls(url, soup)
        saved = download_images_from_urls(url, image_urls) if download_images else []
        return ExhibitRecord(
            url=url,
            title=title,
            period=period,
            section_type=section_type,
            section_text=section_text,
            image_urls=image_urls,
            saved_images=saved,
        )
    finally:
        if own:
            s.close()


In [14]:
# %% [엔드투엔드 크롤러]
def crawl_maru_current(list_url: str = LIST_URL, *, max_pages: int = MAX_PAGES, limit: Optional[int] = None, download_images: bool = DOWNLOAD_IMAGES) -> List[ExhibitRecord]:
    detail_urls = list_current_detail_urls(list_url, max_pages=max_pages)
    if limit is not None:
        detail_urls = detail_urls[:limit]
    results: List[ExhibitRecord] = []
    with get_session() as s:
        _ = _get_soup(list_url, s)
        for du in detail_urls:
            try:
                rec = parse_detail(du, s, download_images=download_images)
                results.append(rec)
            except Exception as e:
                results.append(ExhibitRecord(
                    url=du, title="", period="", section_type="", section_text=f"[ERROR] {e}", image_urls=[], saved_images=[]
                ))
            time.sleep(SLEEP_BETWEEN)
    return results


In [15]:
# %% [실행 예시]
# 필요시 설정 변경 후 실행
DOWNLOAD_IMAGES = False  # True로 바꾸면 이미지 저장
MAX_PAGES = 1

records = crawl_maru_current(LIST_URL, max_pages=MAX_PAGES, limit=10, download_images=DOWNLOAD_IMAGES)
df = pd.DataFrame([{**asdict(r),
                    "images_count": len(r.image_urls),
                    "first_image": r.image_urls[0] if r.image_urls else "",
                    "saved_count": len(r.saved_images)} for r in records])

# 주요 컬럼 미리보기
cols = ["title", "period", "section_type", "images_count", "first_image", "url"]
display(df[cols])

# 저장(선택)
df.to_csv("maru_current_exhibits.csv", index=False, encoding="utf-8-sig")
df.to_json("maru_current_exhibits.json", orient="records", force_ascii=False)
print("Saved maru_current_exhibits.(csv|json)")


,title,period,section_type,images_count,first_image,url
0,제70회 창작미술협회전,2025.10.15-10.20,,5,https://maruartcenter.co.kr/bizdemo133414/comp...,https://maruartcenter.co.kr/default/exhibit/ex...
1,"만화, 4·3과 민주주의를 그리다展",2025.10.15-10.20,본문,4,https://maruartcenter.co.kr/bizdemo133414/comp...,https://maruartcenter.co.kr/default/exhibit/ex...
2,전경애展,2025.10.15-10.20,본문,3,https://maruartcenter.co.kr/bizdemo133414/comp...,https://maruartcenter.co.kr/default/exhibit/ex...
3,제51회 영토회 領土會,2025.10.15-10.20,본문,4,https://maruartcenter.co.kr/bizdemo133414/comp...,https://maruartcenter.co.kr/default/exhibit/ex...
4,예원예술대학교 만화게임영상학과 졸업전시ㅣGAGAMO GALLERY,2025.10.15-10.20,본문,4,https://maruartcenter.co.kr/bizdemo133414/comp...,https://maruartcenter.co.kr/default/exhibit/ex...
5,아주 아틀리에 전시ㅣ회심,2025.10.15-10.20,,3,https://maruartcenter.co.kr/bizdemo133414/comp...,https://maruartcenter.co.kr/default/exhibit/ex...


Saved maru_current_exhibits.(csv|json)


In [16]:
# %% [디버그 도우미 - 테이블 구조 확인]
# 한 개 상세글의 TR별 TD 텍스트를 확인해 라벨/값 구조를 눈으로 점검할 수 있습니다.
if 'records' in globals() and records:
    test_url = records[0].url
    with get_session() as _s:
        sp = _get_soup(test_url, _s, referrer=LIST_URL)
        print("[DEBUG] ", test_url)
        for tr in sp.select("tr")[:20]:
            cells = [_clean_text(td.get_text(" ")) for td in tr.find_all("td")]
            if cells:
                print(cells)


[DEBUG]  https://maruartcenter.co.kr/default/exhibit/exhibit01.php?com_board_basic=read_form&com_board_idx=769&sub=01&&com_board_search_code=&com_board_search_value1=&com_board_search_value2=&com_board_page=&&com_board_id=10&&com_board_id=10
['제목 제70회 창작미술협회전 기간 2025.10.15-10.20 제70회 창작미술협회전 ▼ 2025.10.15 - 10.20 마루아트센터 신관 B1층 특별관', '제목', '제70회 창작미술협회전', '기간', '2025.10.15-10.20', '제70회 창작미술협회전 ▼ 2025.10.15 - 10.20 마루아트센터 신관 B1층 특별관', '', '', '', '']
['제목', '제70회 창작미술협회전']
['기간', '2025.10.15-10.20']
['제70회 창작미술협회전 ▼ 2025.10.15 - 10.20 마루아트센터 신관 B1층 특별관']
['']
['', '', '']
['']
